<a href="https://colab.research.google.com/github/BrianKani/GIS-CropForecast/blob/master/data3landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap
import ee

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
import time
import numpy as np
import pandas as pd

In [ ]:
def export_oneimage(img, folder, name, region, scale, crs):
    task = ee.batch.Export.image(img, name, {
        'driveFolder': folder,
        'driveFileNamePrefix': name,
        'region': region,
        'scale': scale,
        'crs': crs
    })
    task.start()
    while task.status()['state'] == 'RUNNING':
        print('Running...')
        # Perhaps task.cancel() at some point.
        time.sleep(10)
    print('Done.', task.status())

locations = pd.read_csv('/content/locations.csv')

# Define the bounding box coordinates for Uasin Gishu County
uasin_gishu_bbox = [[35.3, 0.6], [35.4, 0.6], [35.4, 0.7], [35.3, 0.7]]

# Transforms an Image Collection with 1 band per Image into a single Image with items as bands
# Author: Brian Kipsang

def appendBand(current, previous):
    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous, None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

imgcoll = ee.ImageCollection('MODIS/051/MCD12Q1') \
    .filterBounds(ee.Geometry.Polygon(uasin_gishu_bbox))
img = imgcoll.iterate(appendBand)

for loc1, loc2, lat, lon in locations.values:
    fname = '{}_{}'.format(int(loc1), int(loc2))

    offset = 0.11
    scale = 500
    crs = 'EPSG:4326'

    region = str([
        [lat - offset, lon + offset],
        [lat + offset, lon + offset],
        [lat + offset, lon - offset],
        [lat - offset, lon - offset]])

    while True:
        try:
            export_oneimage(img, 'Data_mask', fname, region, scale, crs)
        except:
            print('retry')
            time.sleep(10)
            continue
        break
